In [1]:
#04/02/2019
import numpy as np # Algebre linéaire
import pandas as pd # Data processing, CSV

from PIL import Image # Image display
import matplotlib.pyplot as plt

from io import BytesIO # Image request
import requests

import bq_helper # Data train querry
from bq_helper import BigQueryHelper

import os

In [2]:
def images_from_url(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

In [3]:
open_images = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="open_images")
bq_assistant = BigQueryHelper("bigquery-public-data", "open_images")

# Liste des tables :
#['annotations_bbox', 'dict', 'images', 'labels']

Using Kaggle's public dataset BigQuery integration.
Using Kaggle's public dataset BigQuery integration.


In [12]:
subQueryImages = """
(SELECT image_id, original_url
FROM `bigquery-public-data.open_images.images`)
"""

subQueryBox = """
(SELECT image_id, label_name, x_min, x_max, y_min, y_max
FROM `bigquery-public-data.open_images.annotations_bbox`
ORDER BY image_id
LIMIT 100000)
"""

subQueryWord = """
(SELECT label_name, label_display_name
FROM `bigquery-public-data.open_images.dict`)
"""

labelQuery = """
(SELECT ROW_NUMBER() OVER (ORDER BY wrd.label_name) - 1 AS row, wrd.label_name
FROM (SELECT DISTINCT(label_name) FROM """ + subQueryBox + """) lab
JOIN """ + subQueryWord + """ wrd ON lab.label_name = wrd.label_name)"""

mainQuery = """
SELECT img.original_url, wrd.label_display_name, box.label_name, idx.row, box.x_min, box.x_max, box.y_min, box.y_max
FROM """ + subQueryImages + """ img
INNER JOIN """ + subQueryBox + """ box ON img.image_id = box.image_id
INNER JOIN """ + subQueryWord + """ wrd ON box.label_name = wrd.label_name
INNER JOIN """ + labelQuery + """ idx ON idx.label_name = wrd.label_name
ORDER BY img.image_id
"""

dataset = open_images.query_to_pandas_safe(mainQuery)

"""
for i in range(0, 10):
    plt.imshow(images_from_url(dataset.original_url[i]))
    plt.show()
    print(dataset.label_display_name[i])
    print(dataset.original_url[i])"""

'\nfor i in range(0, 10):\n    plt.imshow(images_from_url(dataset.original_url[i]))\n    plt.show()\n    print(dataset.label_display_name[i])\n    print(dataset.original_url[i])'

In [16]:
print(dataset.row)

0        508
1        147
2        450
3         68
4        147
5        306
6        306
7        189
8         96
9        423
10       226
11       423
12       173
13       450
14       226
15       141
16       544
17        96
18       508
19        68
20       450
21       325
22       159
23        68
24       291
25       286
26       251
27       492
28       557
29       300
        ... 
99970    390
99971    584
99972     68
99973    492
99974    423
99975    300
99976    502
99977    480
99978    300
99979    423
99980    300
99981    291
99982    262
99983    475
99984    417
99985      3
99986      0
99987      0
99988    349
99989    543
99990     49
99991    399
99992    454
99993     27
99994     14
99995     15
99996    203
99997    343
99998    404
99999    555
Name: row, Length: 100000, dtype: int64


In [ ]:
"""
file = images_from_url(dataset.original_url[120])

tmp = np.array(file.resize((100, 100)))
plt.imshow(tmp)
plt.show()
print(dataset.label_display_name[120])
"""

features = []
labels = []

# dataset.label_name.size
data_length = 10

for data_ex in range (0, data_length):
    features.append(np.array(images_from_url(dataset.original_url[data_ex]).resize((100, 100))))
    labels.append([[dataset.label_name[data_ex]],[dataset.x_min[data_ex], dataset.y_min[data_ex], dataset.x_max[data_ex], dataset.y_max[data_ex]]])

In [ ]:
for tmp in range(0, 10):
    plt.imshow(features[tmp])
    plt.show()
    print(labels[tmp])